In [1]:
# Implementation of https://www.jstor.org/stable/25734098
# Bayes Bayes Bayes

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import pymc as pm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

C:\Users\theal\AppData\Local\Temp\ipykernel_29680\2146678780.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the dataset
df_raw = pd.read_csv('./data/train_with_dummies.csv', index_col=[0])

# Specify prefixes of columns to drop
prefixes_to_drop = ['Id', 'SaleType', 'SaleCondition', 'SalePrice']

# Drop specified columns before imputation
df_filtered = df_raw.drop([col for col in df_raw.columns if any(col.startswith(prefix) for prefix in prefixes_to_drop)], axis=1)

# Impute missing values in the filtered dataset
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_filtered), columns=df_filtered.columns)

# Extract the SalePrice column from the original dataset for use as the target variable
sale_price_col = df_raw['SalePrice']
sale_price_mean = np.mean(sale_price_col)

# Scale the imputed dataset
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_imputed)
df_scaled = pd.DataFrame(scaled_data, columns=df_imputed.columns)

# Define data_x and data_y for model input
data_x = df_scaled

selected_columns = [
    'OverallQual',
    'MasVnrArea',
    'TotalBsmtSF',
    'GrLivArea',
    'GarageCars',
    'Neighborhood_NWAmes',
    'Neighborhood_NoRidge',
    'Neighborhood_NridgHt',
    'Exterior1st_CBlock',
    'ExterQual_TA',
    'Foundation_PConc',
    'BsmtFinType1_GLQ',
    'KitchenQual_Ex',
    'KitchenQual_TA',
    'GarageType_BuiltIn'
]

# Select only the specified columns for model input
data_x_selected = df_scaled[selected_columns]

data_y = sale_price_col.reset_index(drop=True)  # Reset index to ensure alignmen

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data_x_selected, data_y, test_size=0.2, random_state=42)

In [4]:
n_features = X_train.shape[1]
with pm.Model() as model:
    # Priors
    lambda_ = pm.HalfCauchy('lambda_', beta=1, shape=n_features)
    tau = pm.HalfCauchy('tau', beta=1)
    beta = pm.Normal('beta', mu=0, sigma=tau * lambda_, shape=n_features)
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=pm.math.dot(X_train, beta), sigma=0.1, observed=y_train)
    
    # Inference
    trace = pm.sample(1000, tune=1000, return_inferencedata=True, chains=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [lambda_, tau, beta]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 350 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [5]:
# Access beta samples from the posterior for predictions
beta_samples = trace.posterior['beta'].values
mean_beta = np.mean(beta_samples, axis=(0, 1))  # Average over both chain and draw dimensions

# Predict y values for X_test using mean_beta
y_pred = np.dot(X_test, mean_beta)

# (Optional) Evaluate predictions
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Squared Error: {mse}')

Root Mean Squared Error: 187910.01890554023
Mean Squared Error: 35310175205.08048


In [ ]:
# Assuming selected_columns is a list of your feature names corresponding to X_train
selected_columns = [
    # Your feature names here, matching the order in X_train
]

# Create a dictionary mapping feature names to their mean beta coefficient
feature_coefficients = dict(zip(selected_columns, mean_beta))

# Display the feature coefficients
for feature, coef in feature_coefficients.items():
    print(f"{feature}: {coef}")